In [ ]:
# Основные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Установка стиля графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Модели и метрики
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, 
    confusion_matrix, classification_report,
    roc_curve, precision_recall_curve, auc,
    ConfusionMatrixDisplay, PrecisionRecallDisplay, RocCurveDisplay
)
from sklearn.inspection import permutation_importance
from sklearn.metrics import average_precision_score

# Baseline модели
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

# Модели недели 6
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import (
    RandomForestClassifier, 
    AdaBoostClassifier, 
    GradientBoostingClassifier,
    StackingClassifier
)

print("Библиотеки успешно импортированы")

In [ ]:
# Загрузка данных
url = "https://raw.githubusercontent.com/Ilya190206/aie-group/refs/heads/main/homeworks/HW06/S06-hw-dataset-01.csv"
df = pd.read_csv(url)

print("Данные успешно загружены")
print(f"Размер датасета: {df.shape}")
print(f"Колонки: {df.columns.tolist()}")

In [ ]:
# Просмотр первых строк
print("Первые 5 строк датасета:")
display(df.head())

print("\n" + "="*50)
print("Информация о данных:")
df.info()

print("\n" + "="*50)
print("Статистические характеристики числовых признаков:")
display(df.describe())

# Проверка пропусков
print("\n" + "="*50)
print("Пропуски в данных:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])
if missing_values.sum() == 0:
    print("Пропусков нет!")

# Распределение целевой переменной
print("\n" + "="*50)
print("Распределение целевой переменной (target):")
target_dist = df['target'].value_counts()
print(target_dist)
print(f"\nДоли классов: {target_dist / len(df)}")

# Визуализация распределения таргета
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Гистограмма
axes[0].bar(['Class 0', 'Class 1'], target_dist.values, color=['skyblue', 'salmon'])
axes[0].set_title('Распределение классов', fontsize=14)
axes[0].set_ylabel('Количество')
axes[0].set_xlabel('Класс')

# Круговая диаграмма
axes[1].pie(target_dist.values, labels=['Class 0', 'Class 1'], 
           autopct='%1.1f%%', colors=['skyblue', 'salmon'])
axes[1].set_title('Доли классов', fontsize=14)

plt.tight_layout()
plt.savefig('artifacts/figures/target_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Анализ категориальных признаков
cat_features = ['cat_contract', 'cat_region', 'cat_payment']
print("Анализ категориальных признаков:")
print("-" * 40)

for feature in cat_features:
    unique_vals = df[feature].unique()
    print(f"\n{feature}:")
    print(f"  Уникальные значения: {sorted(unique_vals)}")
    print(f"  Количество уникальных значений: {len(unique_vals)}")
    
    # Распределение значений
    value_counts = df[feature].value_counts().sort_index()
    print(f"  Распределение: {value_counts.to_dict()}")
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Барплот
    axes[0].bar(value_counts.index.astype(str), value_counts.values)
    axes[0].set_title(f'Распределение {feature}', fontsize=14)
    axes[0].set_xlabel(feature)
    axes[0].set_ylabel('Количество')
    
    # Барплот по классам
    cross_tab = pd.crosstab(df[feature], df['target'])
    cross_tab.plot(kind='bar', ax=axes[1])
    axes[1].set_title(f'{feature} по классам', fontsize=14)
    axes[1].set_xlabel(feature)
    axes[1].set_ylabel('Количество')
    axes[1].legend(['Class 0', 'Class 1'])
    
    plt.tight_layout()
    plt.savefig(f'artifacts/figures/{feature}_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()

# Анализ числовых признаков
print("\n" + "="*50)
print("Анализ числового признака tenure_months:")
print(f"Минимум: {df['tenure_months'].min()}")
print(f"Максимум: {df['tenure_months'].max()}")
print(f"Среднее: {df['tenure_months'].mean():.2f}")
print(f"Медиана: {df['tenure_months'].median()}")

# Визуализация tenure_months
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Гистограмма
axes[0].hist(df['tenure_months'], bins=20, edgecolor='black', alpha=0.7)
axes[0].set_title('Распределение tenure_months', fontsize=14)
axes[0].set_xlabel('tenure_months')
axes[0].set_ylabel('Частота')

# Боксплот по классам
data = [df[df['target'] == 0]['tenure_months'], 
        df[df['target'] == 1]['tenure_months']]
axes[1].boxplot(data, labels=['Class 0', 'Class 1'])
axes[1].set_title('tenure_months по классам', fontsize=14)
axes[1].set_ylabel('tenure_months')

plt.tight_layout()
plt.savefig('artifacts/figures/tenure_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Корреляционная матрица числовых признаков (первые 10 для наглядности)
num_features = [f'num{i:02d}' for i in range(1, 25)] + ['tenure_months']
plt.figure(figsize=(12, 10))
correlation_matrix = df[num_features].corr()
sns.heatmap(correlation_matrix, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица числовых признаков', fontsize=16)
plt.tight_layout()
plt.savefig('artifacts/figures/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Создание папок для артефактов
Path("artifacts/figures").mkdir(parents=True, exist_ok=True)

# Определение признаков и целевой переменной
X = df.drop(['id', 'target'], axis=1)
y = df['target']

print(f"Размер X: {X.shape}")
print(f"Размер y: {y.shape}")
print(f"Признаки: {X.columns.tolist()}")
print(f"Количество признаков: {len(X.columns)}")

# Разделение на train/test с фиксированным random_state и стратификацией
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\n" + "="*50)
print("Разделение данных:")
print(f"Размер X_train: {X_train.shape}")
print(f"Размер X_test: {X_test.shape}")
print(f"Размер y_train: {y_train.shape}")
print(f"Размер y_test: {y_test.shape}")

# Проверка распределения классов в train и test
print("\nРаспределение классов в train:")
print(y_train.value_counts(normalize=True))
print("\nРаспределение классов в test:")
print(y_test.value_counts(normalize=True))

# Обоснование выбора параметров
print("\n" + "="*50)
print("Обоснование выбора параметров:")
print("1. test_size=0.2: Стандартное значение для сохранения достаточного количества данных для обучения")
print("2. random_state=42: Для воспроизводимости результатов")
print("3. stratify=y: Для сохранения распределения классов в train и test наборах")
print("4. Разделение проводится ДО любых преобразований для избежания data leakage")

In [ ]:
print("="*50)
print("BASELINE МОДЕЛИ")
print("="*50)

# 1. Dummy Classifier
print("\n1. Dummy Classifier (стратегия: most_frequent):")
dummy = DummyClassifier(strategy='most_frequent', random_state=42)
dummy.fit(X_train, y_train)
y_pred_dummy = dummy.predict(X_test)
y_pred_proba_dummy = dummy.predict_proba(X_test)[:, 1]

dummy_metrics = {
    'accuracy': accuracy_score(y_test, y_pred_dummy),
    'f1': f1_score(y_test, y_pred_dummy),
    'roc_auc': roc_auc_score(y_test, y_pred_proba_dummy)
}

print(f"Accuracy: {dummy_metrics['accuracy']:.4f}")
print(f"F1-score: {dummy_metrics['f1']:.4f}")
print(f"ROC-AUC: {dummy_metrics['roc_auc']:.4f}")

# 2. Logistic Regression
print("\n2. Logistic Regression с StandardScaler:")
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(random_state=42, max_iter=1000))
])

lr_pipeline.fit(X_train, y_train)
y_pred_lr = lr_pipeline.predict(X_test)
y_pred_proba_lr = lr_pipeline.predict_proba(X_test)[:, 1]

lr_metrics = {
    'accuracy': accuracy_score(y_test, y_pred_lr),
    'f1': f1_score(y_test, y_pred_lr),
    'roc_auc': roc_auc_score(y_test, y_pred_proba_lr)
}

print(f"Accuracy: {lr_metrics['accuracy']:.4f}")
print(f"F1-score: {lr_metrics['f1']:.4f}")
print(f"ROC-AUC: {lr_metrics['roc_auc']:.4f}")

# Сравнение baseline моделей
print("\n" + "="*50)
print("СРАВНЕНИЕ BASELINE МОДЕЛЕЙ")
print("-" * 40)

comparison_df = pd.DataFrame({
    'Model': ['Dummy', 'Logistic Regression'],
    'Accuracy': [dummy_metrics['accuracy'], lr_metrics['accuracy']],
    'F1-score': [dummy_metrics['f1'], lr_metrics['f1']],
    'ROC-AUC': [dummy_metrics['roc_auc'], lr_metrics['roc_auc']]
})

display(comparison_df)

# Визуализация ROC-кривых для baseline моделей
plt.figure(figsize=(10, 6))

# Dummy
fpr_dummy, tpr_dummy, _ = roc_curve(y_test, y_pred_proba_dummy)
roc_auc_dummy = auc(fpr_dummy, tpr_dummy)
plt.plot(fpr_dummy, tpr_dummy, label=f'Dummy (AUC = {roc_auc_dummy:.3f})', linestyle='--')

# Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred_proba_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {roc_auc_lr:.3f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривые: Baseline модели')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

plt.savefig('artifacts/figures/baseline_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nИнтерпретация baseline моделей:")
print("1. Dummy Classifier показывает минимально достижимые метрики")
print("2. Logistic Regression должен показать лучшие результаты за счет учета взаимосвязей признаков")
print("3. ROC-AUC > 0.5 показывает, что модель лучше случайного угадывания")

In [ ]:
print("="*50)
print("DECISION TREE CLASSIFIER")
print("="*50)

# Определение параметров для GridSearch
dt_param_grid = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ['gini', 'entropy'],
    'ccp_alpha': [0.0, 0.01, 0.1]  # Параметр для cost-complexity pruning
}

dt = DecisionTreeClassifier(random_state=42)

print("Подбор гиперпараметров через GridSearchCV (5-fold CV)...")
dt_grid = GridSearchCV(
    dt, dt_param_grid, 
    cv=5, 
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

dt_grid.fit(X_train, y_train)

print("\nЛучшие параметры:")
print(dt_grid.best_params_)
print(f"Лучший CV ROC-AUC: {dt_grid.best_score_:.4f}")

# Обучение лучшей модели
best_dt = dt_grid.best_estimator_
y_pred_dt = best_dt.predict(X_test)
y_pred_proba_dt = best_dt.predict_proba(X_test)[:, 1]

dt_metrics = {
    'accuracy': accuracy_score(y_test, y_pred_dt),
    'f1': f1_score(y_test, y_pred_dt),
    'roc_auc': roc_auc_score(y_test, y_pred_proba_dt),
    'best_params': dt_grid.best_params_,
    'cv_score': dt_grid.best_score_
}

print("\nМетрики на тестовой выборке:")
print(f"Accuracy: {dt_metrics['accuracy']:.4f}")
print(f"F1-score: {dt_metrics['f1']:.4f}")
print(f"ROC-AUC: {dt_metrics['roc_auc']:.4f}")

# Визуализация дерева (упрощенная версия для лучшего понимания)
plt.figure(figsize=(20, 10))
plot_tree(best_dt, 
          feature_names=X.columns,
          class_names=['Class 0', 'Class 1'],
          filled=True, 
          rounded=True,
          max_depth=3,  # Показываем только 3 уровня для читаемости
          fontsize=10)
plt.title(f'Decision Tree (max_depth={best_dt.max_depth})', fontsize=16)
plt.savefig('artifacts/figures/decision_tree_structure.png', dpi=300, bbox_inches='tight')
plt.show()

# Анализ важности признаков
feature_importance_dt = pd.DataFrame({
    'feature': X.columns,
    'importance': best_dt.feature_importances_
}).sort_values('importance', ascending=False)

print("\nТоп-10 важных признаков по Decision Tree:")
display(feature_importance_dt.head(10))

# Визуализация важности признаков
plt.figure(figsize=(12, 6))
top_features = feature_importance_dt.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Важность признака')
plt.title('Важность признаков (Decision Tree)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('artifacts/figures/dt_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

# Анализ влияния глубины дерева на качество
print("\nАнализ влияния глубины дерева:")
depth_results = []
for depth in [1, 2, 3, 5, 7, 10, 15, 20, None]:
    dt_temp = DecisionTreeClassifier(max_depth=depth, random_state=42)
    dt_temp.fit(X_train, y_train)
    train_score = roc_auc_score(y_train, dt_temp.predict_proba(X_train)[:, 1])
    test_score = roc_auc_score(y_test, dt_temp.predict_proba(X_test)[:, 1])
    depth_results.append({
        'max_depth': depth if depth else 'None',
        'train_roc_auc': train_score,
        'test_roc_auc': test_score
    })

depth_df = pd.DataFrame(depth_results)
print(depth_df)

# График зависимости качества от глубины
plt.figure(figsize=(10, 6))
plt.plot(depth_df['max_depth'].astype(str), depth_df['train_roc_auc'], 
         marker='o', label='Train ROC-AUC', linewidth=2)
plt.plot(depth_df['max_depth'].astype(str), depth_df['test_roc_auc'], 
         marker='s', label='Test ROC-AUC', linewidth=2)
plt.xlabel('Максимальная глубина дерева')
plt.ylabel('ROC-AUC')
plt.title('Зависимость качества от глубины дерева')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('artifacts/figures/dt_depth_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nВыводы по Decision Tree:")
print("1. Дерево показывает склонность к переобучению при большой глубине")
print("2. GridSearchCV помог найти оптимальные гиперпараметры")
print("3. Важность признаков дает первую интуицию о влиянии разных факторов")

In [ ]:
print("="*50)
print("RANDOM FOREST CLASSIFIER")
print("="*50)

# Определение параметров для GridSearch
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=42, n_jobs=-1)

print("Подбор гиперпараметров через GridSearchCV (5-fold CV)...")
rf_grid = GridSearchCV(
    rf, rf_param_grid, 
    cv=5, 
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

rf_grid.fit(X_train, y_train)

print("\nЛучшие параметры:")
print(rf_grid.best_params_)
print(f"Лучший CV ROC-AUC: {rf_grid.best_score_:.4f}")

# Обучение лучшей модели
best_rf = rf_grid.best_estimator_
y_pred_rf = best_rf.predict(X_test)
y_pred_proba_rf = best_rf.predict_proba(X_test)[:, 1]

rf_metrics = {
    'accuracy': accuracy_score(y_test, y_pred_rf),
    'f1': f1_score(y_test, y_pred_rf),
    'roc_auc': roc_auc_score(y_test, y_pred_proba_rf),
    'best_params': rf_grid.best_params_,
    'cv_score': rf_grid.best_score_
}

print("\nМетрики на тестовой выборке:")
print(f"Accuracy: {rf_metrics['accuracy']:.4f}")
print(f"F1-score: {rf_metrics['f1']:.4f}")
print(f"ROC-AUC: {rf_metrics['roc_auc']:.4f}")

# Важность признаков для Random Forest
feature_importance_rf = pd.DataFrame({
    'feature': X.columns,
    'importance': best_rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nТоп-10 важных признаков по Random Forest:")
display(feature_importance_rf.head(10))

# Визуализация важности признаков
plt.figure(figsize=(12, 6))
top_features_rf = feature_importance_rf.head(15)
plt.barh(range(len(top_features_rf)), top_features_rf['importance'])
plt.yticks(range(len(top_features_rf)), top_features_rf['feature'])
plt.xlabel('Важность признака')
plt.title('Важность признаков (Random Forest)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('artifacts/figures/rf_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

# Сравнение с Decision Tree
print("\nСравнение важности признаков (Decision Tree vs Random Forest):")
importance_comparison = pd.DataFrame({
    'feature': X.columns,
    'dt_importance': best_dt.feature_importances_,
    'rf_importance': best_rf.feature_importances_
})
importance_comparison['diff'] = abs(importance_comparison['dt_importance'] - importance_comparison['rf_importance'])

print("\nПризнаки с наибольшей разницей в важности:")
display(importance_comparison.sort_values('diff', ascending=False).head(10))

# Анализ влияния количества деревьев
print("\nАнализ влияния количества деревьев на качество Random Forest:")
n_estimators_results = []
for n_trees in [10, 20, 50, 100, 200, 300]:
    rf_temp = RandomForestClassifier(
        n_estimators=n_trees, 
        max_depth=best_rf.max_depth,
        min_samples_leaf=best_rf.min_samples_leaf,
        max_features=best_rf.max_features,
        random_state=42,
        n_jobs=-1
    )
    rf_temp.fit(X_train, y_train)
    train_score = roc_auc_score(y_train, rf_temp.predict_proba(X_train)[:, 1])
    test_score = roc_auc_score(y_test, rf_temp.predict_proba(X_test)[:, 1])
    n_estimators_results.append({
        'n_estimators': n_trees,
        'train_roc_auc': train_score,
        'test_roc_auc': test_score
    })

n_estimators_df = pd.DataFrame(n_estimators_results)
print(n_estimators_df)

# График зависимости качества от количества деревьев
plt.figure(figsize=(10, 6))
plt.plot(n_estimators_df['n_estimators'], n_estimators_df['train_roc_auc'], 
         marker='o', label='Train ROC-AUC', linewidth=2)
plt.plot(n_estimators_df['n_estimators'], n_estimators_df['test_roc_auc'], 
         marker='s', label='Test ROC-AUC', linewidth=2)
plt.xlabel('Количество деревьев')
plt.ylabel('ROC-AUC')
plt.title('Зависимость качества Random Forest от количества деревьев')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('artifacts/figures/rf_n_estimators_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nВыводы по Random Forest:")
print("1. Random Forest показывает лучшую обобщающую способность по сравнению с одним деревом")
print("2. Ансамблирование уменьшает variance и улучшает устойчивость к переобучению")
print("3. Random Forest дает более стабильные оценки важности признаков")

In [ ]:
print("="*50)
print("GRADIENT BOOSTING CLASSIFIER")
print("="*50)

# Определение параметров для GridSearch
gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5, 10]
}

gb = GradientBoostingClassifier(random_state=42)

print("Подбор гиперпараметров через GridSearchCV (5-fold CV)...")
gb_grid = GridSearchCV(
    gb, gb_param_grid, 
    cv=5, 
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

gb_grid.fit(X_train, y_train)

print("\nЛучшие параметры:")
print(gb_grid.best_params_)
print(f"Лучший CV ROC-AUC: {gb_grid.best_score_:.4f}")

# Обучение лучшей модели
best_gb = gb_grid.best_estimator_
y_pred_gb = best_gb.predict(X_test)
y_pred_proba_gb = best_gb.predict_proba(X_test)[:, 1]

gb_metrics = {
    'accuracy': accuracy_score(y_test, y_pred_gb),
    'f1': f1_score(y_test, y_pred_gb),
    'roc_auc': roc_auc_score(y_test, y_pred_proba_gb),
    'best_params': gb_grid.best_params_,
    'cv_score': gb_grid.best_score_
}

print("\nМетрики на тестовой выборке:")
print(f"Accuracy: {gb_metrics['accuracy']:.4f}")
print(f"F1-score: {gb_metrics['f1']:.4f}")
print(f"ROC-AUC: {gb_metrics['roc_auc']:.4f}")

# Важность признаков для Gradient Boosting
feature_importance_gb = pd.DataFrame({
    'feature': X.columns,
    'importance': best_gb.feature_importances_
}).sort_values('importance', ascending=False)

print("\nТоп-10 важных признаков по Gradient Boosting:")
display(feature_importance_gb.head(10))

# Визуализация важности признаков
plt.figure(figsize=(12, 6))
top_features_gb = feature_importance_gb.head(15)
plt.barh(range(len(top_features_gb)), top_features_gb['importance'])
plt.yticks(range(len(top_features_gb)), top_features_gb['feature'])
plt.xlabel('Важность признака')
plt.title('Важность признаков (Gradient Boosting)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('artifacts/figures/gb_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

# Learning curve для Gradient Boosting
print("\nАнализ кривой обучения Gradient Boosting:")
train_sizes = np.linspace(0.1, 1.0, 10)
train_scores = []
test_scores = []

for size in train_sizes:
    n_samples = int(size * len(X_train))
    X_train_subset = X_train.iloc[:n_samples]
    y_train_subset = y_train.iloc[:n_samples]
    
    gb_temp = GradientBoostingClassifier(
        n_estimators=best_gb.n_estimators,
        learning_rate=best_gb.learning_rate,
        max_depth=best_gb.max_depth,
        random_state=42
    )
    
    gb_temp.fit(X_train_subset, y_train_subset)
    
    train_score = roc_auc_score(y_train_subset, gb_temp.predict_proba(X_train_subset)[:, 1])
    test_score = roc_auc_score(y_test, gb_temp.predict_proba(X_test)[:, 1])
    
    train_scores.append(train_score)
    test_scores.append(test_score)

# График кривой обучения
plt.figure(figsize=(10, 6))
plt.plot(train_sizes * len(X_train), train_scores, marker='o', label='Train ROC-AUC', linewidth=2)
plt.plot(train_sizes * len(X_train), test_scores, marker='s', label='Test ROC-AUC', linewidth=2)
plt.xlabel('Размер обучающей выборки')
plt.ylabel('ROC-AUC')
plt.title('Кривая обучения Gradient Boosting')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('artifacts/figures/gb_learning_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nВыводы по Gradient Boosting:")
print("1. Gradient Boosting часто показывает лучшие результаты за счет последовательного улучшения")
print("2. Boosting хорошо справляется со сложными нелинейными зависимостями")
print("3. Требует тщательного подбора learning rate и других гиперпараметров")